<a href="https://colab.research.google.com/github/Art-phys/Lesson_HF_LR/blob/main/Lesson_HF_RL_Unit8_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Блок 8: Проксимальный (Ближайший) градиент политики (PPO) с помощью PyTorch 🤖

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit9/thumbnail.png" alt="Unit 8" width="70%"/>


В этом блокноте вы научитесь **кодировать своего агента PPO с нуля с помощью PyTorch, используя реализацию CleanRL в качестве модели**.

Чтобы проверить его надежность, мы собираемся обучить его в среде:

- [LunarLander-v2 🚀](https://www.gymlibrary.dev/environments/box2d/lunar_lander/)


⬇️ Вот пример того, чего вы достигнете. ⬇️

In [1]:
%%html
<video controls autoplay><source src="https://huggingface.co/sb3/ppo-LunarLander-v2/resolve/main/replay.mp4" type="video/mp4"></video>

## Цели этого блокнота 🏆

В конце записной книжки вы:

- Сможете **закодировать своего агента PPO с нуля с помощью PyTorch**.
- Сможете **отправить своего обученного агента и код в центр** с хорошим видеоповтором и оценочным баллом 🔥.